# rsyncファイルダウンロード
## 目的
- リモートサーバから、rsyncで JupyterHubのディレクトリにダウンロードする

# モジュールのインポート

In [ ]:
"""
目的：標準モジュール、外部ライブラリ、および自作モジュールのインポート確認。
使い方：プログラム開始時に必ず最初に実行する。
引数：なし
返り値：なし（インポートエラー時は ImportError を送出）
"""
import os
import sys
import json
from datetime import datetime

# libディレクトリを検索パスに追加
current_dir = os.getcwd()
lib_path = os.path.join(current_dir, 'lib')
if lib_path not in sys.path:
    sys.path.append(lib_path)

# チェック対象モジュール
modules = [
    ('pexpect', '外部ライブラリ'),
    ('rsync_utils', '自作同期モジュール'),
    ('logger_config', '自作ロガー設定')
]

print(f"{'Module':<15} | {'Status':<8}")
print("-" * 30)

all_ok = True
for mod, desc in modules:
    try:
        __import__(mod)
        print(f"{mod:<15} | OK")
    except ImportError:
        print(f"{mod:<15} | MISSING")
        all_ok = False

if not all_ok:
    raise ImportError("モジュールの読み込みに失敗しました。libフォルダを確認してください。")

# 自作関数のインポート
from rsync_utils import rsync_pull
print("\n[INFO] インポート完了: rsync_pull が利用可能です。")

# ユーザー事前定義

In [ ]:
"""
目的：vault/config.jsonを読込。ファイル不在時は雛形を作成し、実行を停止させる。
使い方：停止後、作成された vault/config.json を編集してからセルを再実行する。
引数：なし
返り値：なし（SystemExitで後続処理をブロック）
"""
CONFIG_PATH = os.path.join(os.getcwd(), "vault", "config.json")

def init_config(path):
    """
    目的：設定ファイルの初期化。存在しなければ雛形を作成する。
    引数：path (str): 設定ファイルの保存先。
    返り値：bool: ファイルが存在すればTrue、新規作成ならFalse。
    """
    if not os.path.exists(path):
        os.makedirs(os.path.dirname(path), exist_ok=True)
        default_val = {
            "REMOTE_USER": "sysope", "REMOTE_HOST": "192.168.3.64",
            "PASSWORD": "P!ssw0rd1234", "SRC_DIR": "/home/sysope/rsync_pull_files/",
            "DEST_DIR": "result", "BW_LIMIT": "1024", "EXT_FILTER": "*.csv",
            "LOG_FILE_NAME": "result/execute.log", "LOG_TO_FILE": True, "ENABLE_LOG": False
        }
        with open(path, 'w', encoding='utf-8') as f:
            json.dump(default_val, f, indent=4, ensure_ascii=False)
        return False
    return True

if not init_config(CONFIG_PATH):
    raise SystemExit(f"[STOP] {CONFIG_PATH} を作成しました。設定を編集後に再実行してください。")
else:
    with open(CONFIG_PATH, 'r', encoding='utf-8') as f:
        custom_config = json.load(f)
    if not os.path.isabs(custom_config["DEST_DIR"]):
        custom_config["DEST_DIR"] = os.path.join(os.getcwd(), custom_config["DEST_DIR"])
    print(f"[INFO] 設定ロード成功: {custom_config['REMOTE_HOST']}")

# ファイル同期（ダウンロード）

In [ ]:
"""
目的：rsync_pullを実行し、新しくダウンロードされたファイルのみをログに記録する。
使い方：セル1・セル2が正常終了した状態で実行する。
引数：custom_config (dict): vault/config.json から読み込まれた設定。
返り値：なし（実行結果を画面に表示）
"""
try:
    print(f"[{datetime.now().strftime('%H:%M:%S')}] 同期処理を開始します...")
    
    # セル1でインポートした関数を実行
    result = rsync_pull(custom_config)
    
    print(f"\n最終ステータス: {result}")
    
except Exception as e:
    print(f"[ERROR] エラーが発生しました: {e}")